<h1> Using Machine Learning APIs </h1>

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Copy-paste your API Key here:

In [ ]:
# Use the chown command to change the ownership of repository to user
!sudo chown -R jupyter:jupyter /home/jupyter/gcp-training

In [ ]:
APIKEY="YOUR_API_KEY"  # Replace with your API key

<b> Note: Make sure you generate an API Key and replace the value above. The sample key will not work.</b>


<h2> Invoke Translate API </h2>

In [ ]:
# Running the Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# Use the service
inputs = ['is it really this easy?', 'amazing technology', 'wow']
outputs = service.translations().list(source='en', target='fr', q=inputs).execute()

# Print outputs
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

In [ ]:
# TODO: przetestować translacje pomiędzy kilkoma językami poprzez podmiane parametrów w metodzie wywołaniu API 
# (np: pl -> de, en -> es, ...)

<h2> Invoke Vision API </h2>

The Vision API can work off an image in Cloud Storage or embedded directly into a POST message. I'll use Cloud Storage and do OCR on this image: <img src="https://storage.googleapis.com/cloud-training-demos/vision/sign2.jpg" width="200" />.  That photograph is from http://www.publicdomainpictures.net/view-image.php?image=15842


In [ ]:
# Running the Vision API
import base64
import json
IMAGE="gs://cloud-training-demos/vision/sign2.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)

# Output the `responses`
# print(json.dumps(responses, indent=4))
print(responses['responses'][0]['textAnnotations'][0])

In [ ]:
foreigntext = responses['responses'][0]['textAnnotations'][0]['description']
foreignlang = responses['responses'][0]['textAnnotations'][0]['locale']

# Let's output the `foreignlang` and `foreigntext`
print(foreignlang, foreigntext)

<h2> Translate sign </h2>

In [ ]:
inputs=[foreigntext]
outputs = service.translations().list(source=foreignlang, target='en', q=inputs).execute()

# Print the outputs
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

In [ ]:
# TODO:
# 1. Pobrac z internetu obrazek na którym jest osoba z widoczną twarzą
# 2. Stworzyć Cloud Storage bucket (Konsola -> Cloud Storage -> Create) (nazwa musi byc globalnie unikalna)
# 3. Choose how to control access to objects -> Odznaczyc: Enforce public access prevention on this bucket -> Create
# 4. Permissions -> Grant access -> New principals -> allUsers
# 5. Role -> Cloud Storage Object Viewer
# 6. Wrzucić pobrany obrazek
# 7. Wejść w plik, skopiować do schowka gsutil URI i wkleić do zmiennej IMAGE poniżej

In [ ]:
IMAGE = 'YOUR_GS_URI'

In [ ]:
# 8. Korzystając z powyzszego przykladu wywolania API Vision AI oraz dokumentacji API:
# https://cloud.google.com/vision/docs/reference/rest/v1/Feature
# napisać skrypt który będzie wykrywał twarze na pobranym obrazku
# 9. Rozparsować odpowiedź API tak aby otrzymać odpowiedź czy osoba na zdjeciu jest usmiechnieta wedlug API

In [ ]:
request = vservice.images().annotate(body={
    #...
        })
responses = request.execute(num_retries=3)

# Printowanie odpowiedzi dla 'faceAnnotations'
# print(json.dumps(responses['responses'][0]['faceAnnotations'], indent=4))

<h2> Sentiment analysis with Language API </h2>

Let's evaluate the sentiment of some famous quotes using Google Cloud Natural Language API.

In [ ]:
# Evaluating the sentiment with Google Cloud Natural Language API
lservice = build('language', 'v1beta1', developerKey=APIKEY)
quotes = [
  'To succeed, you must have tremendous perseverance, tremendous will.',
  'It’s not that I’m so smart, it’s just that I stay with problems longer.',
  'Love is quivering happiness.',
  'Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.',
  'What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?',
  'When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. '
]
for text in quotes:
# The `documents.analyzeSentiment` method analyzes the sentiment of the provided text.
  response = lservice.documents().analyzeSentiment(
    body={
      'document': {
         'type': 'PLAIN_TEXT',
         'content': text
      }
    }).execute()
  polarity = response['documentSentiment']['polarity']
  magnitude = response['documentSentiment']['magnitude']

# Lets output the value of each `polarity`, `magnitude` and `quote`
  print('POLARITY=%s MAGNITUDE=%s for %s' % (polarity, magnitude, quote))

In [ ]:
# TODO: napisac skrypt ktory znajdzie i przeanalizuje encje w ponizszym tekscie:
text = 'President Trump will speak from the White House, located at 1600 Pennsylvania Ave NW, Washington, DC, on October 7.'

In [ ]:
# Hint: Uzyj metody `documents.analyzeEntities`
# https://cloud.google.com/natural-language/docs/reference/rest/v1beta2/Entity
response = ...
print(response)

In [2]:
# TODO: wyprintowac nazwe encji, typ, waznosc (salience) oraz znalezione przez api metadane.

In [ ]:
for entity in response['entities']:
    print('...')

<h2> Speech API </h2>

The Speech API can work on streaming data, audio content encoded and embedded directly into the POST message, or on a file on Cloud Storage. Here I'll pass in this <a href="https://storage.googleapis.com/cloud-training-demos/vision/audio.raw">audio file</a> in Cloud Storage.

In [ ]:
# Using the Speech API by passing audio file as an argument
sservice = build('speech', 'v1', developerKey=APIKEY)
# The `speech.recognize` method performs synchronous speech recognition.
# It receive results after all audio has been sent and processed.
response = sservice.speech().recognize(
    body={
        'config': {
            'languageCode' : 'en-US',
            'encoding': 'LINEAR16',
            'sampleRateHertz': 16000
        },
        'audio': {
            'uri': 'gs://cloud-training-demos/vision/audio.raw'
            }
        }).execute()

# Let's output the value of `response`
print(response)

In [ ]:
print(response['results'][0]['alternatives'][0]['transcript'])

# Let's output the value of `'Confidence`
print('Confidence=%f' % response['results'][0]['alternatives'][0]['confidence'])